---
title: 'Reshaping Data with melt() and pivot()'
---





## Packages


In [ ]:
import pandas as pd

## Data

Run the code below to load and define the datasets to be used in this lesson.


In [ ]:
# Temperatures dataset
temperatures = pd.DataFrame({
    "country": ["Sweden", "Denmark", "Norway"],
    "1994": [1, 2, 3],
    "1995": [3, 4, 5],
    "1996": [5, 6, 7],
})

# Fuels Wide dataset
fuels_wide = pd.read_csv(
    "https://raw.githubusercontent.com/the-graph-courses/idap_book/main/data/oil_per_capita_wide.csv"
)

# Eurostat Births Wide dataset
euro_births_wide = pd.read_csv(
    "https://raw.githubusercontent.com/the-graph-courses/idap_book/main/data/euro_births_wide.csv"
)

# Contracts dataset
contracts = pd.read_csv(
    "https://raw.githubusercontent.com/the-graph-courses/idap_book/main/data/chicago_contracts_20_23.csv"
)

# Population dataset
population = pd.read_csv(
    "https://raw.githubusercontent.com/the-graph-courses/idap_book/main/data/tidyr_population.csv"
)

## Introduction

Reshaping is a data manipulation technique that involves re-orienting the rows and columns of a dataset. This is often required to make data easier to analyze or understand.

In this lesson, we will cover how to effectively reshape data using `pandas` functions.

## Learning Objectives

- Understand what wide data format is and what long data format is.
- Learn how to reshape wide data to long data using `melt()`.
- Learn how to reshape long data to wide data using `pivot()`.

## What Do "Wide" and "Long" Mean?

The terms "wide" and "long" are best understood in the context of example datasets. Let's take a look at some now.

Imagine that you have three products for which you collect sales data over three months.

**Wide Format:**

| Product | Jan | Feb | Mar |
|---------|-----|-----|-----|
| A       | 100 | 120 | 110 |
| B       | 90  | 95  | 100 |
| C       | 80  | 85  | 90  |

---

**Long Format:**

| Product | Month | Sales |
|---------|-------|-------|
| A       | Jan   | 100   |
| A       | Feb   | 120   |
| A       | Mar   | 110   |
| B       | Jan   | 90    |
| B       | Feb   | 95    |
| B       | Mar   | 100   |
| C       | Jan   | 80    |
| C       | Feb   | 85    |
| C       | Mar   | 90    |

Take a minute to study the two datasets to make sure you understand the relationship between them.

In the **wide dataset**, each observational unit (each product) occupies only one row, and each measurement (sales in Jan, Feb, Mar) is in a separate column.

In the **long dataset**, on the other hand, each observational unit (each product) occupies multiple rows, with one row for each measurement.

---

Here's another example with mock data, where the observational units are countries:

**Long Format:**

| Country | Year | GDP   |
|---------|------|-------|
| USA     | 2020 | 21433 |
| USA     | 2021 | 22940 |
| China   | 2020 | 14723 |
| China   | 2021 | 17734 |

**Wide Format:**

| Country | GDP_2020 | GDP_2021 |
|---------|----------|----------|
| USA     | 21433    | 22940    |
| China   | 14723    | 17734    |

---

The examples above are both time-series datasets because the measurements are repeated across time. But the concepts of wide and long are relevant to other kinds of data too.

Consider the example below, showing the number of employees in different departments of three companies:

**Wide Format:**

| Company | HR  | Sales | IT  |
|---------|-----|-------|-----|
| A       | 10  | 20    | 15  |
| B       | 8   | 25    | 20  |
| C       | 12  | 18    | 22  |

**Long Format:**

| Company | Department | Employees |
|---------|------------|-----------|
| A       | HR         | 10        |
| A       | Sales      | 20        |
| A       | IT         | 15        |
| B       | HR         | 8         |
| B       | Sales      | 25        |
| B       | IT         | 20        |
| C       | HR         | 12        |
| C       | Sales      | 18        |
| C       | IT         | 22        |

In the wide dataset, each observational unit (each company) occupies only one row, with the repeated measurements (number of employees in different departments) spread across multiple columns.

In the long dataset, each observational unit is spread over multiple rows.

::: {.callout-note title="Vocab"}
The **observational units**, sometimes called **statistical units**, are the primary entities or items described by the dataset.

In the first example, the observational units were products; in the second example, countries; and in the third example, companies.
:::

::: {.callout-tip title="Practice"}

## Practice Q: Wide or Long?

Consider the `temperatures` dataset created earlier:


In [ ]:
temperatures

Is this data in a wide or long format?


In [ ]:
# Write your answer here
# The data is in a wide format.

:::

## When Should You Use Wide vs. Long Data?

The truth is, it really depends on what you want to do! The wide format is great for *displaying data* because it's easy to visually compare values this way. Long data is best for some data analysis tasks, like grouping and plotting.

It is essential to know how to switch from one format to the other easily. Switching from the wide to the long format, or the other way around, is called **reshaping**.

## From Wide to Long

To practice reshaping from a wide to a long format, we'll consider data from Our World in Data on fossil fuel consumption per capita. You can find the data [here](https://ourworldindata.org/grapher/fossil-fuels-per-capita).

Below, we view this data on fossil fuel consumption per capita:


In [ ]:
fuels_wide

We observe that each observational unit (each country) occupies only one row, with the repeated measurements of fossil fuel consumption (in kilowatt-hour equivalents) spread out across multiple columns. Hence, this dataset is in a wide format.

To convert it to a long format, we can use the convenient `melt` function. Within `melt`, we define which columns we want to reshape:


In [ ]:
fuels_long = fuels_wide.melt(id_vars=["Entity", "Code"], value_vars=fuels_wide.columns[2:])
fuels_long

Very easy!

Let's break down the code:

- **`id_vars`** refers to the column(s) that will not be reshaped. In this case, it's the "Entity" and "Code" columns.
- **`value_vars`** is the column(s) that will be reshaped. In this case, it's the year columns.

The years are now indicated in the variable `variable`, and all the consumption values occupy a single variable, `value`. We may wish to rename the `variable` column to `year`, and the `value` column to `oil_consumption`. This can be done directly in the `melt` function:


In [ ]:
fuels_long = fuels_wide.melt(
    id_vars=['Entity', 'Code'],
    value_vars=fuels_wide.columns[2:],
    var_name='year',
    value_name='oil_consumption'
)
fuels_long

You may also want to remove the `y_` in front of each year. This can be achieved with a string operation. We'll also arrange the data by country and year:


In [ ]:
fuels_long['year'] = fuels_long['year'].str.replace('y_', '').astype(int)
fuels_long = fuels_long.sort_values(by=['Entity', 'year'])
fuels_long

Here's what we did above:

- Used `str.replace()` to remove the `y_` prefix from each year.
- Converted the `year` column to integers using `astype(int)`.
- Sorted the data by "Entity" and "year" using `sort_values()`.

::: {.callout-tip title="Practice"}

## Practice Q: Temperatures to Long

Convert the `temperatures` dataset created above into a long format. Your answer should have the following column names: **"country"**, **"year"**, and **"avgtemp"**.


In [ ]:
# Your code here
temperatures_long = temperatures.melt(
    id_vars=["country"], var_name="year", value_name="avgtemp"
)
temperatures_long

:::

::: {.callout-tip title="Practice"}

## Practice Q: Eurostat Births to Long

For this practice question, you will use the `euro_births_wide` dataset from [Eurostat](https://ec.europa.eu/eurostat/databrowser/view/tps00204/default/table). It shows the annual number of births in 50 European countries:


In [ ]:
euro_births_wide

The data is in a wide format. Convert it to a long format DataFrame that has the following column names: **"country"**, **"year"**, and **"births_count"**.


In [ ]:
# Your code here
births_long = euro_births_wide.melt(
    id_vars=["country"], var_name="year", value_name="births_count"
)
births_long

:::

## From Long to Wide

Now you know how to reshape from wide to long with `melt()`. How about going the other way, from long to wide? For this, you can use the `pivot()` function.

But before we consider how to use this function to manipulate long data, let's first consider *where* you're likely to run into long data.

While wide data tends to come from external sources (as we have seen above), long data, on the other hand, is likely to be created by *you* while data wrangling, especially in the course of grouped aggregations.

Let's see an example of this now.

We will use a dataset of contracts granted by the city of Chicago from 2020 to 2023. You can find more information about the data [here](https://data.cityofchicago.org/Administration-Finance/Contracts/rsxa-ify5/).


In [ ]:
contracts

Each row corresponds to one contract, and we have each contract's ID number, the year in which it was granted, the amount of the contract, and the vendor's name and address, among other variables.

Now, consider the following grouped summary of the `contracts` dataset, which shows the number of contracts by state of the vendor in each year:


In [ ]:
contracts_summary = contracts.groupby("state")["year"].value_counts().reset_index(name='count')
contracts_summary

The output of this grouped operation is a quintessentially "long" dataset. Each observational unit (each state) occupies multiple rows, with one row for each measurement (each year).

Now, let's see how to convert such long data into a wide format with `pivot()`.

The code is quite straightforward:


In [ ]:
contracts_wide = contracts_summary.pivot(
    index="state", columns="year", values="count"
).reset_index()
contracts_wide

As you can see, `pivot()` has three important arguments:

- **`index`** defines which column(s) to use as the new index. In our case, it's the "state" since we want each row to represent one state.
- **`columns`** identifies which variable to use to define column names in the wide format. In our case, it's the "year". You can see that the years are now the column names.
- **`values`** specifies which values will become the core of the wide data format. In our case, it's the number of contracts "count".

You might also want to have the *years* be your primary observational units, with each year occupying one row. This can be carried out similarly to the above example, but with `year` as the index and `state` as the columns:


In [ ]:
contracts_wide_year = contracts_summary.pivot(
    index="year", columns="state", values="count"
).reset_index()
contracts_wide_year

Here, the unique observation units (our rows) are now the years (2020, 2021, 2022, 2023).

::: {.callout-tip title="Practice"}

## Practice Q: Temperatures back to Wide

Convert the long `temperatures_long` dataset you created above back to a wide format. Your answer should have the following column names: **"country"**, **"1994"**, **"1995"**, and **"1996"**.


In [ ]:
# Your code here
temperatures_wide = temperatures_long.pivot(
    index="country", columns="year", values="avgtemp"
).reset_index()
temperatures_wide

:::

::: {.callout-tip title="Practice"}

## Practice Q: Population to Wide

The `population` dataset shows the populations of 219 countries over time.

Reshape this data into a wide format. Your answer should have 219 rows and multiple columns representing different years.


In [ ]:
population

In [ ]:
# Your code here

# Reshape the long data into wide format
population_wide = population.pivot(
    index="country", columns="year", values="population"
).reset_index()

# Display the wide format data
population_wide

:::

## Reshaping Can Be Hard

We have mostly looked at very simple examples of reshaping here, but in the wild, reshaping can be very difficult to do accurately.

When you run into such cases, we recommend looking at the [official documentation](https://pandas.pydata.org/docs/user_guide/reshaping.html) of reshaping from the pandas team, as it is quite rich in examples.

## Wrap-Up

Congratulations! You've mastered the art of reshaping data with pandas.

You now understand the differences between wide and long formats and can skillfully use `melt()` and `pivot()` to transform your data as needed.

Remember, there's no universally "best" format—it depends on your specific analysis or visualization needs. With these skills, you're now equipped to handle data in any shape it comes in. Keep practicing with different datasets to reinforce your learning!

## Solution Q: Wide or Long?

The data is in a wide format.

## Solution Q: Temperatures to Long


In [ ]:
# Melt the wide data into long format
temperatures_long = temperatures.melt(
    id_vars=["country"], var_name="year", value_name="avgtemp"
)

# Display the long format data
temperatures_long

## Solution Q: Eurostat Births to Long


In [ ]:
# Melt the wide data into long format
births_long = euro_births_wide.melt(
    id_vars=["country"], var_name="year", value_name="births_count"
)

# Display the long format data
births_long

## Solution Q: Temperatures back to Wide


In [ ]:
# Pivot the long data into wide format
temperatures_wide = temperatures_long.pivot(
    index="country", columns="year", values="avgtemp"
).reset_index()

# Display the wide format data
temperatures_wide

## Solution Q: Population to Wide


In [ ]:
# Pivot the long data into wide format
population_wide = population.pivot(
    index="country", columns="year", values="population"
).reset_index()

# Display the wide format data
population_wide